In [14]:
#!/usr/bin/env python3
"""
cmd_ngc_robust.py

Versión robusta para leer .cat de SExtractor (NGC362, NGC6405),
hacer cross-match G,R,I en J2000 y generar CMDs g vs (g-r) y r vs (r-i).

Mejora clave: si astropy lanza TypeError por una incompatibilidad con numpy,
el script captura ese error y usa KDTree (scipy) como fallback automáticamente.
"""
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------- Ajustes ----------
objects = {
    "NGC362": "/home/astrotomi/Desktop/Source-extractor/NGC362",
    "NGC6405": "/home/astrotomi/Desktop/Source-extractor/NGC6405",
}
match_tol_arcsec = 1.0   # tolerancia de emparejado (arcsec). Aumenta si no hay matches
max_magerr = 0.1         # corte por error de magnitud (si existe)
# índices según tu header (0-based)
RA_IDX = 3
DEC_IDX = 4
MAG_BEST_IDX = 5
MAG_AUTO_IDX = 7
MAGERR_AUTO_IDX = 11
FLAGS_IDX = 13
# -----------------------------

def try_import_astropy():
    try:
        from astropy.coordinates import SkyCoord
        from astropy import units as u
        return SkyCoord, u
    except Exception:
        return None, None

def try_import_scipy_kdtree():
    try:
        from scipy.spatial import cKDTree
        return cKDTree
    except Exception:
        return None

def radec_to_xyz(ra_deg, dec_deg):
    """
    Convierte arrays ra,dec (grados) a vectores unitarios 3D (x,y,z).
    Usa np.column_stack para evitar problemas con dtype en vstack/concatenate.
    """
    ra = np.deg2rad(np.asarray(ra_deg, dtype=float))
    dec = np.deg2rad(np.asarray(dec_deg, dtype=float))
    x = np.cos(dec) * np.cos(ra)
    y = np.cos(dec) * np.sin(ra)
    z = np.sin(dec)
    return np.column_stack((x, y, z))

def skycoord_match(cat1_ra, cat1_dec, cat2_ra, cat2_dec, tol_arcsec=1.0):
    """
    Devuelve indices emparejados: (idx1_array, idx2_array)
    idx1_array: índices en cat1; idx2_array: índices en cat2

    Intenta usar astropy; si astropy falla con TypeError (incompatibilidad numpy/astropy),
    cae automáticamente al método KDTree (scipy).
    """
    # asegurar arrays float
    cat1_ra = np.asarray(cat1_ra, dtype=float)
    cat1_dec = np.asarray(cat1_dec, dtype=float)
    cat2_ra = np.asarray(cat2_ra, dtype=float)
    cat2_dec = np.asarray(cat2_dec, dtype=float)

    SkyCoord, u = try_import_astropy()
    if SkyCoord is not None:
        try:
            c1 = SkyCoord(ra=cat1_ra * u.deg, dec=cat1_dec * u.deg)
            c2 = SkyCoord(ra=cat2_ra * u.deg, dec=cat2_dec * u.deg)
            # Este bloque puede lanzar TypeError en algunas combinaciones de numpy/astropy
            idx, d2d, _ = c1.match_to_catalog_sky(c2)
            sep_mask = d2d.arcsec <= tol_arcsec
            print("  Usando astropy SkyCoord.match_to_catalog_sky para emparejado.")
            return np.where(sep_mask)[0], idx[sep_mask]
        except TypeError as e:
            # Capturamos la incompatibilidad y saltamos al fallback KDTree
            print("  WARNING: astropy.match_to_catalog_sky lanzó TypeError (incompatibilidad numpy/astropy).")
            print("           Mensaje:", e)
            print("           Cayendo al método KDTree (scipy) como fallback.")
        except Exception as e:
            # Otros errores de astropy → informar y luego intentar fallback
            print("  WARNING: astropy lanzÓ una excepción al emparejar:", repr(e))
            print("           Intentando fallback KDTree (scipy).")

    # Fallback KDTree en caso de que astropy no esté disponible o falle
    cKDTree = try_import_scipy_kdtree()
    if cKDTree is None:
        raise RuntimeError(
            "Ni astropy (funcional) ni scipy.spatial.cKDTree están disponibles. "
            "Instala 'astropy' (recomendado) o 'scipy' para usar el emparejado angular.\n"
            "Ejemplo: pip install astropy  (o pip install scipy)"
        )

    v1 = radec_to_xyz(cat1_ra, cat1_dec)
    v2 = radec_to_xyz(cat2_ra, cat2_dec)
    tree = cKDTree(v2)
    # convertir tol arcsec a chord distance (rad)
    tol_rad = np.deg2rad(tol_arcsec / 3600.0)
    chord = 2.0 * np.sin(tol_rad / 2.0)
    dists, idx = tree.query(v1, k=1)
    sep_mask = dists <= chord
    print("  Usando scipy.spatial.cKDTree para emparejado (fallback).")
    return np.where(sep_mask)[0], idx[sep_mask]

def read_sextractor_cat(path):
    try:
        df = pd.read_csv(path, comment='#', delim_whitespace=True, header=None, engine='python')
    except Exception as e:
        raise RuntimeError(f"Error leyendo {path}: {e}")
    return df

def extract_mag_column(df):
    mag = None
    magerr = None
    if MAG_AUTO_IDX < df.shape[1]:
        mag = df.iloc[:, MAG_AUTO_IDX].values
    elif MAG_BEST_IDX < df.shape[1]:
        mag = df.iloc[:, MAG_BEST_IDX].values
    else:
        raise RuntimeError("No encuentro columnas MAG_AUTO ni MAG_BEST en el .cat")
    if MAGERR_AUTO_IDX < df.shape[1]:
        magerr = df.iloc[:, MAGERR_AUTO_IDX].values
    return mag, magerr

def basic_quality_mask(df):
    if FLAGS_IDX < df.shape[1]:
        flags = df.iloc[:, FLAGS_IDX].values
        return flags == 0
    else:
        return np.ones(len(df), dtype=bool)

# --------- procesamiento ----------
for name, dirpath in objects.items():
    print(f"\nProcesando {name} en {dirpath}")
    if not os.path.isdir(dirpath):
        print(f"  ERROR: carpeta no encontrada: {dirpath}. Verifica la ruta.")
        continue

    files = glob.glob(os.path.join(dirpath, "*.cat"))
    if len(files) == 0:
        print("  No se encontraron archivos .cat en la carpeta.")
        continue

    # heurística para detectar G,R,I
    g_file = r_file = i_file = None
    for f in files:
        base = os.path.basename(f).lower()
        if base.endswith('g.cat'):
            g_file = f
        elif base.endswith('r.cat'):
            r_file = f
        elif base.endswith('i.cat'):
            i_file = f

    # fallback: busca letra 'g','r','i' en el nombre si no terminó con esas
    if not (g_file and r_file and i_file):
        for f in files:
            b = os.path.basename(f).lower()
            if 'g' in b and g_file is None:
                g_file = f
            elif 'r' in b and r_file is None:
                r_file = f
            elif 'i' in b and i_file is None:
                i_file = f

    print(f"  Archivos detectados: G={g_file}, R={r_file}, I={i_file}")
    if not (g_file and r_file and i_file):
        print("  No están los tres filtros G,R,I detectados automáticamente. Revisa nombres.")
        continue

    dfg = read_sextractor_cat(g_file)
    dfr = read_sextractor_cat(r_file)
    dfi = read_sextractor_cat(i_file)

    # extraer RA/DEC y mags
    rag = dfg.iloc[:, RA_IDX].astype(float).values
    decg = dfg.iloc[:, DEC_IDX].astype(float).values
    rar = dfr.iloc[:, RA_IDX].astype(float).values
    decr = dfr.iloc[:, DEC_IDX].astype(float).values
    rai = dfi.iloc[:, RA_IDX].astype(float).values
    deci = dfi.iloc[:, DEC_IDX].astype(float).values

    magg, magerrg = extract_mag_column(dfg)
    magr, magerrr = extract_mag_column(dfr)
    magi, magerri = extract_mag_column(dfi)

    maskg = basic_quality_mask(dfg)
    maskr = basic_quality_mask(dfr)
    maski = basic_quality_mask(dfi)

    if magerrg is not None:
        maskg = maskg & (magerrg < max_magerr)
    if magerrr is not None:
        maskr = maskr & (magerrr < max_magerr)
    if magerri is not None:
        maski = maski & (magerri < max_magerr)

    idxg_all = np.where(maskg)[0]
    idxr_all = np.where(maskr)[0]
    idxi_all = np.where(maski)[0]

    # match G <-> R
    try:
        idxg_rel, idxr_rel = skycoord_match(rag[idxg_all], decg[idxg_all], rar[idxr_all], decr[idxr_all], tol_arcsec=match_tol_arcsec)
    except RuntimeError as e:
        print(f"  ERROR during matching: {e}")
        continue

    if len(idxg_rel) == 0:
        print(f"  AVISO: 0 matches entre G y R con tol {match_tol_arcsec}\". Intenta aumentar match_tol_arcsec.")
    else:
        print(f"  Matches G-R encontrados: {len(idxg_rel)}")

    idxg_global = idxg_all[idxg_rel] if len(idxg_rel) > 0 else np.array([], dtype=int)
    idxr_global = idxr_all[idxr_rel] if len(idxr_rel) > 0 else np.array([], dtype=int)

    if len(idxg_global) > 0:
        # match (G-R) con I
        try:
            idx_rg_in_i_rel, idxi_rel = skycoord_match(rag[idxg_global], decg[idxg_global], rai[idxi_all], deci[idxi_all], tol_arcsec=match_tol_arcsec)
        except RuntimeError as e:
            print(f"  ERROR during matching (GR to I): {e}")
            continue

        if len(idx_rg_in_i_rel) == 0:
            print(f"  AVISO: 0 matches entre (G-R) y I con tol {match_tol_arcsec}\".")
        else:
            idxg_final = idxg_global[idx_rg_in_i_rel]
            idxr_final = idxr_global[idx_rg_in_i_rel]
            idxi_final = idxi_all[idxi_rel]
            print(f"  Matches G-R-I completos: {len(idxg_final)}")

            table = pd.DataFrame({
                "ra": rag[idxg_final],
                "dec": decg[idxg_final],
                "g_mag": magg[idxg_final],
                "r_mag": magr[idxr_final],
                "i_mag": magi[idxi_final],
            })
            table = table.replace([np.inf, -np.inf], np.nan).dropna(subset=["g_mag", "r_mag", "i_mag"])
            table["g_minus_r"] = table["g_mag"] - table["r_mag"]
            table["r_minus_i"] = table["r_mag"] - table["i_mag"]

            outcsv = os.path.join(dirpath, f"{name}_matched_GRI_tol{match_tol_arcsec:.1f}arcsec.csv")
            table.to_csv(outcsv, index=False)
            print(f"  Tabla emparejada guardada en: {outcsv}")

            # CMD g vs g-r
            plt.figure(figsize=(6,8))
            plt.gca().invert_yaxis()
            plt.scatter(table["g_minus_r"], table["g_mag"], s=6)
            plt.xlabel("g - r")
            plt.ylabel("g")
            plt.title(f"{name} CMD: g vs (g-r)  (N={len(table)})")
            plt.grid(alpha=0.3)
            outpng1 = os.path.join(dirpath, f"{name}_CMD_g_vs_g-r.png")
            plt.savefig(outpng1, dpi=200, bbox_inches='tight')
            plt.close()
            print(f"  Diagrama g vs (g-r) guardado en: {outpng1}")

            # CMD r vs r-i
            plt.figure(figsize=(6,8))
            plt.gca().invert_yaxis()
            plt.scatter(table["r_minus_i"], table["r_mag"], s=6)
            plt.xlabel("r - i")
            plt.ylabel("r")
            plt.title(f"{name} CMD: r vs (r-i)  (N={len(table)})")
            plt.grid(alpha=0.3)
            outpng2 = os.path.join(dirpath, f"{name}_CMD_r_vs_r-i.png")
            plt.savefig(outpng2, dpi=200, bbox_inches='tight')
            plt.close()
            print(f"  Diagrama r vs (r-i) guardado en: {outpng2}")
    else:
        print("  No hay objetos válidos tras el emparejado G<->R; revisa tolerancia o headers.")

print("\nHecho. Revisa los .csv y .png en las carpetas correspondientes.")



Procesando NGC362 en /home/astrotomi/Desktop/Source-extractor/NGC362
  Archivos detectados: G=/home/astrotomi/Desktop/Source-extractor/NGC362/362G.cat, R=/home/astrotomi/Desktop/Source-extractor/NGC362/362R.cat, I=/home/astrotomi/Desktop/Source-extractor/NGC362/362I.cat
           Mensaje: concatenate() got an unexpected keyword argument 'dtype'
           Cayendo al método KDTree (scipy) como fallback.
  Usando scipy.spatial.cKDTree para emparejado (fallback).
  Matches G-R encontrados: 589
           Mensaje: concatenate() got an unexpected keyword argument 'dtype'
           Cayendo al método KDTree (scipy) como fallback.
  Usando scipy.spatial.cKDTree para emparejado (fallback).
  Matches G-R-I completos: 541
  Tabla emparejada guardada en: /home/astrotomi/Desktop/Source-extractor/NGC362/NGC362_matched_GRI_tol1.0arcsec.csv
  Diagrama g vs (g-r) guardado en: /home/astrotomi/Desktop/Source-extractor/NGC362/NGC362_CMD_g_vs_g-r.png
  Diagrama r vs (r-i) guardado en: /home/astrotomi/De